In [2]:
import sys
import platform

# --- Install base Python dependencies ---
!pip install numpy matplotlib tqdm scikit-learn umap-learn plotly

# --- TensorFlow installation (Windows GPU support) ---
# TensorFlow 2.12 is the last version with built-in GPU support on Windows.
# Works with CUDA 11.8 + cuDNN 8.6 (auto-downloaded by pip now).

if platform.system() == "Windows":
    print("Detected Windows — installing TensorFlow 2.12 (GPU supported)")
    !pip install tensorflow==2.12.0
else:
    print("Non-Windows system — installing latest TensorFlow")
    !pip install tensorflow

# --- Verify installation ---
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", len(tf.config.list_physical_devices('GPU')) > 0)


   ---------------------------------------- 0.0/90.1 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/90.1 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/90.1 kB ? eta -:--:--
   ------------------------------------ --- 81.9/90.1 kB 573.4 kB/s eta 0:00:01
   ---------------------------------------- 90.1/90.1 kB 464.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.7 MB 3.4 MB/s eta 0:00:03
   ---------------------------------------- 0.1/8.7 MB 3.4 MB/s eta 0:00:03
   ---------------------------------------- 0.1/8.7 MB 3.4 MB/s eta 0:00:03
    --------------------------------------- 0.1/8.7 MB 853.3 kB/s eta 0:00:11
    --------------------------------------- 0.2/8.7 MB 876.1 kB/s eta 0:00:10
    ---------------------------------

ERROR: Could not find a version that satisfies the requirement tensorflow==2.12.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.12.0


ModuleNotFoundError: No module named 'tensorflow'

In [1]:
# ---- Imports ----
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pathlib
import numpy as np
import matplotlib.pyplot as plt

# ---- Paths ----
# Use raw string (r"...") to avoid escaping backslashes
DATA_ROOT = pathlib.Path(r"C:\Users\marku\Downloads\archive\food-101\food-101")
IMAGE_DIR = DATA_ROOT / "images"

if not IMAGE_DIR.exists():
    raise FileNotFoundError(f"Dataset folder missing. Expected: {IMAGE_DIR}")


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
batch_size = 32
img_size = (224, 224)
seed = 42

# Keras only supports train/validation via validation_split,
# so we load train/val first then manually split val → val/test.

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_DIR,
    validation_split=0.3,        # 70% train, 30% temp
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
)

temp_ds = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_DIR,
    validation_split=0.3,        # this split includes val+test
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
)

# Now split temp_ds into validation (20%) and test (10%)
temp_batches = tf.data.experimental.cardinality(temp_ds).numpy()
val_batches = int(temp_batches * (2/3))

val_ds = temp_ds.take(val_batches)
test_ds = temp_ds.skip(val_batches)

class_names = train_ds.class_names
print("Classes:", len(class_names))


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)


In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [ ]:
def build_model_v1(img_size, num_classes):
    base_model = keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=img_size + (3,),
        weights="imagenet"
    )
    base_model.trainable = False  # freeze backbone for model_v1

    inputs = keras.Input(shape=img_size + (3,))
    x = data_augmentation(inputs)
    x = keras.applications.efficientnet.preprocess_input(x)

    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    return model

model = build_model_v1(img_size, len(class_names))

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
epochs = 5

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


In [ ]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation accuracy: {val_acc:.4f}")


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.4f}")


In [ ]:
def predict_image(path):
    img = keras.preprocessing.image.load_img(path, target_size=img_size)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, 0)

    preds = model.predict(img_array)[0]
    class_idx = np.argmax(preds)
    return class_names[class_idx], preds[class_idx]

# Example
# predict_image("data/food-101/images/apple_pie/123.jpg")
